<a href="https://colab.research.google.com/github/Francisss3/Data-Engineering/blob/main/MongoDB_Data_Engineering_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pymongo[srv] dnspython certifi


In [2]:
import certifi
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

# your Atlas credentials
username = "charbelfrancis03"
password = "hUmsVOJeW3LRlLZ6"

# build the SRV URI
uri = (
    f"mongodb+srv://{username}:{password}"
    "@cluster0.8otd0br.mongodb.net"
    "/?retryWrites=true&w=majority"
)

# create the client using certifi’s CA bundle
client = MongoClient(
    uri,
    server_api=ServerApi("1"),
    tls=True,
    tlsCAFile=certifi.where()
)

# test the connection
try:
    client.admin.command("ping")
    print("✅ Connected to MongoDB Atlas!")
except Exception as e:
    print("❌ Connection failed:", e)


✅ Connected to MongoDB Atlas!


In [3]:
# Cell 3: pick your DB & collection
db = client.ecommerce                # “ecommerce” database (created on first write)
products = db.products               # “products” collection


In [4]:
# Cell 4: insert 3 example docs from PDF
initial_products = [
    {"name": "Laptop",   "price": 1200, "category": "Electronics", "inventory": 10},
    {"name": "T-Shirt",  "price":   20, "category": "Clothing",    "inventory": 50},
    {"name": "Sofa",     "price":  700, "category": "Furniture",   "inventory":  5,
     "reviews": [{"user": "Alice", "rating": 4}]}
]

res = products.insert_many(initial_products)
print(f"Inserted initial docs → {res.inserted_ids}")


Inserted initial docs → [ObjectId('685013346d9362496df4525c'), ObjectId('685013346d9362496df4525d'), ObjectId('685013346d9362496df4525e')]


In [5]:
# Cell 5: a) CREATE – add 5 more products
more_products = [
    {"name": "Headphones", "price": 150, "category": "Electronics", "inventory": 25},
    {"name": "Jeans",      "price":  45, "category": "Clothing",    "inventory": 40},
    {"name": "Dining Set", "price": 900, "category": "Furniture",   "inventory":  8},
    {"name": "Watch",      "price": 300, "category": "Electronics", "inventory": 15,
     "discount": 0.05},
    {"name": "Jacket",     "price": 120, "category": "Clothing",    "inventory": 30,
     "reviews": [{"user": "Bob", "rating": 5}]}
]
res = products.insert_many(more_products)
print(f"Created 5 more → {len(res.inserted_ids)} new documents")

# b) READ – list Electronics with price > $500
print("\n📦 Electronics > $500:")
for p in products.find({"category": "Electronics", "price": {"$gt": 500}}):
    print(f" • {p['name']}: ${p['price']} (stock: {p['inventory']})")

# c) UPDATE – add 10% discount and reduce price on all Furniture
upd = products.update_many(
    {"category": "Furniture"},
    [
      {"$set": {"discount": 0.10}},
      {"$set": {"price": {"$round": [{ "$multiply": ["$price", 0.90] }, 2]}}}
    ]
)
print(f"\n🔧 Applied discount to {upd.modified_count} furniture items")

# d) DELETE – remove everything with inventory < 5
del_res = products.delete_many({"inventory": {"$lt": 5}})
print(f"🗑️  Deleted {del_res.deleted_count} low-inventory products")


Created 5 more → 5 new documents

📦 Electronics > $500:
 • Laptop: $1200 (stock: 10)

🔧 Applied discount to 2 furniture items
🗑️  Deleted 0 low-inventory products


In [6]:
# Cell 6: add a “reviews” array to an existing doc (e.g. T-Shirt)
evolve = products.update_one(
    {"name": "T-Shirt"},
    {"$set": {
        "reviews": [
            {"user": "Charlie", "rating": 3},
            {"user": "Dana",    "rating": 4}
        ]
    }}
)
print(f"\n✏️  Schema evolution on T-Shirt → matched {evolve.matched_count}, modified {evolve.modified_count}")

# And one more example: add a “rating” field to Watch
evolve2 = products.update_one(
    {"name": "Watch"},
    {"$set": {"averageRating": 4.8}}
)
print(f"✏️  Added averageRating to Watch → matched {evolve2.matched_count}, modified {evolve2.modified_count}")



✏️  Schema evolution on T-Shirt → matched 1, modified 1
✏️  Added averageRating to Watch → matched 1, modified 1
